Final Results

* analyzing: Training time, Classification accuracy on validation set, Inference time on validation set

In [1]:
import json

In [2]:
data_dir = '../data'

In [ ]:
dataset_name = 'cifar10'

class_names = [
    'Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 
    'Dog', 'Frog', 'Horse', 'Ship', 'Truck'
]

with open(f'{data_dir}/params.json') as f:
    my_params = json.load(f)[dataset_name]

In [4]:
my_params

{'patch_size': 4,
 'dim': 256,
 'dim_head': 64,
 'mlp_dim': 512,
 'heads': 8,
 'depth': 4,
 'pool': 'cls',
 'dropout': 0.15,
 'emb_dropout': 0.15,
 'la_depth': 1,
 'la_exp': True,
 'lr': 0.001,
 'epochs': 30,
 'gamma': 30,
 'batch_size': 256}